In [ ]:
#prepare for RDS connection
# !pip install psycopg2

In [1]:
# dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, text
import psycopg2
from getpass import getpass
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

/Users/joeymyers/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Set up connection between notebook and database

In [2]:
# create SQL Alchemy connection from RDS
secret = getpass('Enter the secret value: ')

args ={
    'host':"ogdataset.c11hekhsylui.us-west-1.rds.amazonaws.com",
    'port':'5432',
    'database':"og_dataset",
    'user':"attritionProject",
    'password':secret
}

engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{database}".format(**args))
connection = engine.connect()

Enter the secret value: ········


In [3]:
# read in table from sql 
attrition_df = pd.read_sql('SELECT * FROM og_dataset', connection)

In [4]:
# view dataframe
attrition_df.head()

,Age,Attrition,Business Travel,Daily Rate,Department,Distance from Home,Education Level,Education Field,Employee Count,Employee Number,...,Relationship Satisfaction,Standard Hours,Stock Option Level,Total Working Years,Training Times Last Year,Work Life Balance,Years at Company,Years In Current Role,Years Since Last Promotion,Years With Current Manager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


# Preprocess Data

In [5]:
# determine if there are any missing values and data types for columns
attrition_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Age                         1470 non-null   int64 
 1   Attrition                   1470 non-null   object
 2   Business Travel             1470 non-null   object
 3   Daily Rate                  1470 non-null   int64 
 4   Department                  1470 non-null   object
 5   Distance from Home          1470 non-null   int64 
 6   Education Level             1470 non-null   int64 
 7   Education Field             1470 non-null   object
 8   Employee Count              1470 non-null   int64 
 9   Employee Number             1470 non-null   int64 
 10  Environment Satisfaction    1470 non-null   int64 
 11  Gender                      1470 non-null   object
 12  Hourly Rate                 1470 non-null   int64 
 13  Job Involvement             1470 non-null   int6

In [6]:
# column names
attrition_df.columns

Index(['Age', 'Attrition', 'Business Travel', 'Daily Rate', 'Department',
       'Distance from Home', 'Education Level', 'Education Field',
       'Employee Count', 'Employee Number', 'Environment Satisfaction',
       'Gender', 'Hourly Rate', 'Job Involvement', 'Job Level', 'Job Role',
       'Job Satisfaction', 'Marital Status', 'Monthly Income', 'Monthly Rate',
       'Number Companies Worked', 'Over 18', 'Overtime', 'Percent Salary Hike',
       'Performance Rating', 'Relationship Satisfaction', 'Standard Hours',
       'Stock Option Level', 'Total Working Years', 'Training Times Last Year',
       'Work Life Balance', 'Years at Company', 'Years In Current Role',
       'Years Since Last Promotion', 'Years With Current Manager'],
      dtype='object')

In [7]:
# ambiguous/unnecessary columns - only going to keep MonthlyIncome
attrition_df[["Daily Rate", "Hourly Rate", "Monthly Income", "Monthly Rate", "Standard Hours", "Over 18"]]

,Daily Rate,Hourly Rate,Monthly Income,Monthly Rate,Standard Hours,Over 18
0,1102,94,5993,19479,80,Y
1,279,61,5130,24907,80,Y
2,1373,92,2090,2396,80,Y
3,1392,56,2909,23159,80,Y
4,591,40,3468,16632,80,Y
...,...,...,...,...,...,...
1465,884,41,2571,12290,80,Y
1466,613,42,9991,21457,80,Y
1467,155,87,6142,5174,80,Y
1468,1023,63,5390,13243,80,Y


Drop unnecessary columns

In [8]:
# these columns are either unique identifiers, ambiguous or constant thru each column
columns_to_drop = ["Daily Rate", "Hourly Rate", "Monthly Rate", "Standard Hours", "Employee Count", 
                   "Employee Number", "Over 18"]

In [9]:
attrition_df.drop(columns=columns_to_drop, inplace=True)
attrition_df.head()

,Age,Attrition,Business Travel,Department,Distance from Home,Education Level,Education Field,Environment Satisfaction,Gender,Job Involvement,...,Performance Rating,Relationship Satisfaction,Stock Option Level,Total Working Years,Training Times Last Year,Work Life Balance,Years at Company,Years In Current Role,Years Since Last Promotion,Years With Current Manager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,Female,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,Male,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,Male,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,Female,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,Male,3,...,3,4,1,6,3,3,2,2,2,2


In [10]:
# number of columns now down from 35 to 28
len(attrition_df.columns)

28

Send cleaned Dataframe to the database

In [14]:
# Write cleaned attrition to table in RDS, drop index
attrition_df.to_sql('cleaned_data', engine, index=False)

Send cleaned Dataframe to csv for Tableau

In [11]:
attrition_df.to_csv("clean_data.csv", index=False)

# Encoding

Making two separate lists of columns that need to be encoded and join them

In [15]:
# making a list of columns with object data types
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat

['Attrition',
 'Business Travel',
 'Department',
 'Education Field',
 'Gender',
 'Job Role',
 'Marital Status',
 'Overtime']

In [16]:
# number of unique items in each column
attrition_df[attrition_cat].nunique()

Attrition          2
Business Travel    3
Department         3
Education Field    6
Gender             2
Job Role           9
Marital Status     3
Overtime           2
dtype: int64

In [17]:
# remove attrition from this list as it is our target variable
attrition_cat.remove("Attrition")
attrition_cat

['Business Travel',
 'Department',
 'Education Field',
 'Gender',
 'Job Role',
 'Marital Status',
 'Overtime']

In [18]:
# columns that were pre-encoded with a ranking system that need to be encoded as well
ranking_system_col = ["Job Involvement", "Job Level", "Job Satisfaction", "Work Life Balance", 
                      "Relationship Satisfaction", "Environment Satisfaction", "Stock Option Level",
                     "Education Level", "Performance Rating"]

In [19]:
attrition_df[ranking_system_col]

,Job Involvement,Job Level,Job Satisfaction,Work Life Balance,Relationship Satisfaction,Environment Satisfaction,Stock Option Level,Education Level,Performance Rating
0,3,2,4,1,1,2,0,2,3
1,2,2,2,3,4,3,1,1,4
2,2,1,3,3,2,4,0,2,3
3,3,1,3,3,3,4,0,4,3
4,3,1,2,3,4,1,1,1,3
...,...,...,...,...,...,...,...,...,...
1465,4,2,4,3,3,3,1,2,3
1466,2,3,1,3,1,4,1,1,3
1467,4,2,2,3,2,2,1,3,4
1468,2,2,2,2,4,4,0,3,3


In [20]:
# join lists together to be encoded
attrition_cat = attrition_cat + ranking_system_col
attrition_cat

['Business Travel',
 'Department',
 'Education Field',
 'Gender',
 'Job Role',
 'Marital Status',
 'Overtime',
 'Job Involvement',
 'Job Level',
 'Job Satisfaction',
 'Work Life Balance',
 'Relationship Satisfaction',
 'Environment Satisfaction',
 'Stock Option Level',
 'Education Level',
 'Performance Rating']

In [22]:
# amount of columns that will be encoded: 16
len(attrition_cat)

16

Encode Data

In [23]:
# MOVE TO TOP OF ML MODEL?
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

C:\Users\gilig\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Business Travel_Non-Travel,Business Travel_Travel_Frequently,Business Travel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,Education Field_Human Resources,Education Field_Life Sciences,Education Field_Marketing,Education Field_Medical,...,Stock Option Level_1,Stock Option Level_2,Stock Option Level_3,Education Level_1,Education Level_2,Education Level_3,Education Level_4,Education Level_5,Performance Rating_3,Performance Rating_4
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


Merge OneHotEncoded dataframe with original

In [24]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

C:\Users\gilig\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,Attrition,Distance from Home,Monthly Income,Number Companies Worked,Percent Salary Hike,Total Working Years,Training Times Last Year,Years at Company,Years In Current Role,...,Stock Option Level_1,Stock Option Level_2,Stock Option Level_3,Education Level_1,Education Level_2,Education Level_3,Education Level_4,Education Level_5,Performance Rating_3,Performance Rating_4
0,41,Yes,1,5993,8,11,8,0,6,4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,49,No,8,5130,1,23,10,3,10,7,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,37,Yes,2,2090,6,15,7,3,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,33,No,3,2909,1,11,8,3,8,7,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,27,No,2,3468,9,12,6,3,2,2,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


Apply label encoder on the target variable we are trying to predict - convert Yes/No to 1/0

In [25]:
# find the amount of Yes/No values in the Attrition columns
attrition_df["Attrition"].value_counts()

No     1233
Yes     237
Name: Attrition, dtype: int64

In [26]:
# apply label encoder on target variable "Attrition"
enc = LabelEncoder()

attrition_df["Attrition"]= enc.fit_transform(attrition_df["Attrition"])

In [27]:
# look at dataframe to confirm encoding was successful
attrition_df.head()

,Age,Attrition,Distance from Home,Monthly Income,Number Companies Worked,Percent Salary Hike,Total Working Years,Training Times Last Year,Years at Company,Years In Current Role,...,Stock Option Level_1,Stock Option Level_2,Stock Option Level_3,Education Level_1,Education Level_2,Education Level_3,Education Level_4,Education Level_5,Performance Rating_3,Performance Rating_4
0,41,1,1,5993,8,11,8,0,6,4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,49,0,8,5130,1,23,10,3,10,7,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,37,1,2,2090,6,15,7,3,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,33,0,3,2909,1,11,8,3,8,7,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,27,0,2,3468,9,12,6,3,2,2,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [32]:
# Write encoded attrition dataframe to table in RDS, drop index
attrition_df.to_sql('encoded_data', engine, index=False, method='multi')